In [3]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# 3
В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [49]:
data_a = np.zeros(42)
data_a[:10] = 1
data_b = np.zeros(16)
data_b[:4] = 1

In [50]:
conf_interval_a = proportion_confint(10, 34, method = 'wilson')
conf_interval_b = proportion_confint(4, 16, method = 'wilson')

In [51]:
print '95%% confidence interval for probability, a: [%f, %f]' % conf_interval_a
print '95%% confidence interval for probability, b [%f, %f]' % conf_interval_b

95% confidence interval for probability, a: [0.168346, 0.461689]
95% confidence interval for probability, b [0.101821, 0.494983]


In [52]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [53]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [54]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [55]:
print "95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(data_a, data_b)

95% confidence interval for a difference between proportions: [-0.260117, 0.236307]


В вопросе предлагается посчитать достигаемый уровень значимости при альтернативе заразительности зевоты. Какому соотношению долей в двух выборках соответствует эта альтернатива?

In [56]:
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(data_a, data_b))

p-value: 0.924557


In [57]:
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(data_a, data_b), 'less')

p-value: 0.462278


In [62]:
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(data_a, data_b), 'greater')

p-value: 0.537722


In [16]:
from sklearn.cross_validation import train_test_split

# 4
Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

логистическая регрессия по признакам X1,X2,X3;
логистическая регрессия по признакам X4,X5,X6.
Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

In [38]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [39]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [17]:
data = pd.read_csv('./banknotes.txt', sep = '\t')

In [18]:
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [26]:
train_data_X, test_data_X, train_data_y, test_data_y = \
    train_test_split(data[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']], data['real'], train_size=0.75, random_state=1)

In [20]:
from sklearn.linear_model import LogisticRegression

In [28]:
clf1 = LogisticRegression().fit(train_data_X[['X1', 'X2', 'X3']].values, train_data_y.values)
clf1_score = clf1.score(test_data_X[['X1', 'X2', 'X3']].values, test_data_y.values)
clf1_score

0.80000000000000004

In [35]:
clf1_delta_arr = np.zeros(len(test_data_y.values))
for i, y in enumerate(test_data_X[['X1', 'X2', 'X3']].values):
    if clf1.predict(y) != test_data_y.values[i]:
        clf1_delta_arr[i] = 1
sum(clf1_delta_arr)/len(test_data_y.values)

/home/scitator/.conda/envs/python2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/scitator/.conda/envs/python2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/scitator/.conda/envs/python2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape

0.20000000000000001

In [29]:
clf2 = LogisticRegression().fit(train_data_X[['X4', 'X5', 'X6']].values, train_data_y.values)
clf2_score = clf2.score(test_data_X[['X4', 'X5', 'X6']].values, test_data_y.values)
clf2_score

0.97999999999999998

In [36]:
clf2_delta_arr = np.zeros(len(test_data_y.values))
for i, y in enumerate(test_data_X[['X4', 'X5', 'X6']].values):
    if clf2.predict(y) != test_data_y.values[i]:
        clf2_delta_arr[i] = 1
sum(clf2_delta_arr)/len(test_data_y.values)

/home/scitator/.conda/envs/python2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/scitator/.conda/envs/python2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/scitator/.conda/envs/python2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape

0.02

In [40]:
proportions_diff_z_test(proportions_diff_z_stat_rel(clf1_delta_arr, clf2_delta_arr))

0.0032969384555543435

# 5
В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

In [41]:
print "95%% confidence interval for a difference between proportions: [%f, %f]" \
      % proportions_diff_confint_rel(clf1_delta_arr, clf2_delta_arr)

95% confidence interval for a difference between proportions: [0.059945, 0.300055]


# 6
Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [42]:
delta = 525
se = 100
n1 = 200000
delta2 = 541.4
n2 = 100

In [44]:
t = (delta2 - delta)/(se/np.sqrt(n2))
t

1.6399999999999977

In [45]:
p = 1-stats.norm.cdf(t)
p

0.050502583474103968

# 7
Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [46]:
delta = 525
se = 100
n1 = 200000
delta2 = 541.5
n2 = 100

In [47]:
t = (delta2 - delta)/(se/np.sqrt(n2))
t

1.6499999999999999

In [48]:
p = 1-stats.norm.cdf(t)
p

0.049471468033648103